In [1]:
import random
import string
import numpy as np
random.seed()

# Import text from the file

In [2]:
f = open('edgar_allan_poe.txt')
text = f.read()

# Format text into list of lines, lowercase and punctuation removed

In [3]:
lines_formatted = []
lines = text.splitlines()
for line in lines:
    if line:
        lines_formatted.append(line.lower().translate(str.maketrans('', '', string.punctuation)))



# Iterate through lines to populate pi and A dictionaries:

Pi is a dictionary where the keys are the words and the values are the probabilities.

A is a dictionary where the keys are the words and the values are further dictionaries, containing probabilities for following words.

Loop through each line in all input text, and in each line loop through each word:


*   If the word is in the first position in the line, check if it exists in Pi dictionary. If so, increment its value by 1/N where N is number of lines. If not, add it to the dictionary with value 1/N.
*   If the word is not in the first position:
*   Check if the previous word exists in A dictionary. If so, check if current word exists in subdictionary. If so, increment its value by 1. If current word does not exist in subdictionary, add it with value 1. If previous word does not exist in A dictionary, add it and add current word to subdictionary with value 1.
*   For each word in A dictionary, sum up total of values in subdictionary, and divide each number by that count, such that each one sums to 1.







In [4]:
pi = {}
A = {}
words = []
#sum = 0

for line in lines_formatted:
    words = line.split(' ')
    if '' in words:
        words.remove('')
    for i in range(len(words)):
        if i == 0: #if this is the first word in the line
            if words[i] in pi: #if it exists in the pi dictionary
                pi[words[i]] += 1/len(lines_formatted)
            else: #if it does not exist in the pi dictionary
                pi.update({words[i]: 1/len(lines_formatted)})
        else: #if this is not the first word in the line
            if words[i-1] in A: #if the previous word exists in the A dictionary
                if words[i] in A[words[i-1]]: #if the current word exists in the previous word's subdictionary
                    A[words[i-1]][words[i]] += 1
                else: #if the current word does not exist in the subdictionary
                    A[words[i-1]].update({words[i]: 1})
            else:
                A.update({words[i-1]: {words[i]: 1}})

for i in pi:
    #pi[i] = np.log(pi[i])
    pass

for i in A:
    word_count = sum(A[i].values())
    for j in A[i]:
        A[i][j] = A[i][j]/word_count
        #A[i][j] = np.log(A[i][j])


# Generate Text using pi and A

Now generate a poem. For each line, generate a first word using pi, then generate words following it using A. If encountering a word which doesn't exist in A (that is, it only exists in the training text as the end of a line) then add a full stop and stop generating.

generateLine and generatePoem will be functions allowing variable line length and line number.

In [5]:
def generateLine(line_length):
    random_line = []

    words = list(pi.keys())
    weights = list(pi.values())
    random_line.append(random.choices(words,weights)[0])

    for i in range(line_length-1):
        if random_line[i] not in A:
            random_line.append('.')
            break
        words = list(A[random_line[i]].keys())
        weights = list(A[random_line[i]].values())
        random_line.append(random.choices(words,weights)[0])
    return " ".join(random_line).replace(' .','.')

In [6]:
def generatePoem(line_length, line_number):
    random_poem = []
    for _ in range(line_number):
        random_poem.append(generateLine(line_length))
    return "\n".join(random_poem)

In [ ]:
print(generatePoem(10,4))

# Potential improvements

To make the model much better an improvement could be to transform the model into a second-order Markov model. In theory if all words and possibilities are considered then this increases the size of A from M^2 to M^3, however since in this case A only contains word sequences that actually occur in the training data it may be significantly reduced, although with a small set of training data it would likely be restricted by the limited number of possibilities.